In [2]:
%load_ext lab_black

In [3]:
import os

os.chdir("/cerea_raid/users/dumontj/dev/coco2/inv")
import numpy as np
import sys
import pandas as pd
import xarray as xr
import tensorflow as tf
from tensorflow import keras
from dataclasses import dataclass, field
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from icecream import ic
import matplotlib_functions
from mpl_toolkits.axes_grid1 import make_axes_locatable
from models.my_efficientnet import EfficientNet
from models.my_essential_inversors import (
    essential_regressor,
    essential_regressor_2,
    essential_regressor_3,
    linear_regressor,
)
from models.my_mobilenet import MobileNet
from models.my_shufflenet import ShuffleNet
from models.my_squeezenet import SqueezeNet
from models.preprocessing import CloudsLayer, ConditionalNoiseLayer

import include.callbacks as callbacks
import include.generators as generators
import include.loss as loss
import include.optimisers as optimisers
import models.seg as sm
from Data import Data_train
from models.preprocessing import CloudsLayer
from saver import Saver
import logging
import sys
from model_training import Trainer

import hydra
import wandb
from omegaconf import DictConfig, OmegaConf

from include.callbacks import initiate_wb
from model_training import Model_training_manager

from typing import List, Optional

matplotlib_functions.setMatplotlibParam()
plt.viridis()
import joblib
import pickle
from include.generators import Generator
from saver import Saver
from include.callbacks import get_modelcheckpoint, ExtraValidation

import model_eval
from Data import Data_train, Data_eval
import wandb
from wandb.keras import WandbCallback, WandbMetricsLogger
import models.reg as rm

import hydra
from hydra import initialize, compose
from omegaconf import OmegaConf

2023-12-05 17:30:44.846777: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 17:30:44.846816: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 17:30:44.846838: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 17:30:44.853633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
wandb: ERROR Unable to patch Tensorflow/Ke

<Figure size 1920x1440 with 0 Axes>

In [4]:
from models.preprocessing import (
    ConditionalNoiseLayer,
    CloudsLayer,
    TrainingTimeNormalization,
)

# First test

In [24]:
norm_layer = TrainingTimeNormalization(axis=-1, name="preproc_norm")
cloud_layer = CloudsLayer()
noise_layer = ConditionalNoiseLayer([True, False, False])

In [25]:
inputs = tf.keras.layers.Input([64, 64, 3], name="input_layer")
x = cloud_layer(inputs)
x = noise_layer(x)
outputs = norm_layer(x)

model = tf.keras.Model(inputs, outputs)

In [26]:
model.save("test.keras")

In [28]:
model = tf.keras.models.load_model(
    "test.keras",
    custom_objects={
        "TrainingTimeNormalization": TrainingTimeNormalization(
            axis=-1, name="preproc_norm"
        ),
        "CloudsLayer": CloudsLayer(),
        "ConditionalNoiseLayer": ConditionalNoiseLayer([True, False, False]),
    },
    compile=False,
)

In [29]:
model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 64, 64, 3)]       0         
                                                                 
 clouds_layer_4 (CloudsLaye  (None, 64, 64, 3)         0         
 r)                                                              
                                                                 
 conditional_noise_layer_1   (None, 64, 64, 3)         0         
 (ConditionalNoiseLayer)                                         
                                                                 
 preproc_norm (TrainingTime  (None, 64, 64, 3)         0         
 Normalization)                                                  
                                                                 
Total params: 0 (0.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_____________________________

# Second test

In [4]:
import models.reg as rm

In [5]:
model_builder = rm.Reg_model_builder()

In [6]:
model = model_builder.get_model()

In [7]:
model.save("test.keras")

In [8]:
model = tf.keras.models.load_model(
    "test.keras",
    custom_objects={
        "TrainingTimeNormalization": TrainingTimeNormalization(
            axis=-1, name="preproc_norm"
        ),
        "CloudsLayer": CloudsLayer(),
        "ConditionalNoiseLayer": ConditionalNoiseLayer([True, False, False]),
    },
    compile=False,
)

# Third test

In [5]:
# Compose the configuration
cfg = compose(
    config_name="config",
    overrides=[
        "training.max_epochs=10",
        "+experiment=inv_lip",
        "data.path.train.name=2km_Lip",
        "data.path.train.nc=valid_dataset.nc",
        "data.path.valid.name=2km_Lip",
    ],
)

# Print the configuration
print(OmegaConf.to_yaml(cfg))

data:
  init:
    _target_: Data.Data_train
    path_train_ds: ${data.path.directory}/${data.path.train.name}/${data.path.train.nc}
    path_valid_ds: ${data.path.directory}/${data.path.valid.name}/${data.path.valid.nc}
    path_extra_valid_ds: ${data.path.directory}/${data.path.extra_valid.name}/${data.path.extra_valid.nc}
  input:
    dir_seg_models: /cerea_raid/users/dumontj/dev/coco2/dl/res/models
    chan_0: xco2
    chan_1: v_wind
    chan_2: u_wind
    chan_3: no2
    chan_4: None
    clouds: true
    path_clouds_array: /libre/dumontj/coco2/dl-input/clouds/extracted_clouds.npy
  output:
    N_emissions: 1
  path:
    directory: /libre/dumontj/coco2/dl-input
    train:
      name: 2km_Lip
      nc: valid_dataset.nc
    valid:
      name: 2km_Lip
      nc: valid_dataset.nc
    extra_valid:
      name: 2km_Lip
      nc: valid_dataset.nc
dir_res: /libre/dumontj/coco2/weights/new/
exp_name: inv_lip
seed: 42
sweep: true
model:
  type: inversion
  name: essential
  loss_func: MeanAbsol

In [6]:
initiate_wb(cfg)
model_trainer = Model_training_manager(cfg)

wandb: Currently logged in as: joffreydumont (cerea-daml). Use `wandb login --relogin` to force relogin


/libre/dumontj/coco2/dl-input/2km_Lip/valid_dataset.nc


2023-12-05 17:30:57.628009: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14628 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:65:00.0, compute capability: 7.5
2023-12-05 17:30:58.290256: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
ic| self.train.shape: (1152, 64, 64, 4)
wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


In [7]:
model = model_trainer.model
data = model_trainer.data

In [8]:
model.save("test.keras")

In [10]:
model = tf.keras.models.load_model(
    "test.keras",
    custom_objects={
        "TrainingTimeNormalization": TrainingTimeNormalization(
            axis=-1, name="preproc_norm"
        ),
        "CloudsLayer": CloudsLayer(),
        "ConditionalNoiseLayer": ConditionalNoiseLayer([True, False, False]),
    },
    compile=False,
)

In [11]:
tf.keras.models.load_model(
    "test.keras",
    custom_objects={
        "ConditionalNoiseLayer": ConditionalNoiseLayer(
            model_trainer.data.x.xco2_noisy_chans
        ),
        "CloudsLayer": CloudsLayer(
            model_trainer.data.x.clouds_training, model_trainer.data.x.cloudy_chans
        ),
        "TrainingTimeNormalization": TrainingTimeNormalization(
            axis=-1, name="preproc_norm"
        ),
    },
    compile=False,
)

wandb: Waiting for W&B process to finish... (success).


# Fourth test

In [13]:
class TrainingTimeNormalization(tf.keras.layers.Layer):
    def __init__(self, axis=-1, **kwargs):
        super(TrainingTimeNormalization, self).__init__(**kwargs)
        self.normalization = tf.keras.layers.Normalization(axis=axis)

    def build(self, input_shape):
        """Build the normalization layer."""
        self.normalization.build(input_shape)

    def adapt(self, data):
        """Adapts the normalization layer to the data."""
        self.normalization.adapt(data)

    def call(self, x, training=None):
        """Apply or skip normalization based on the training flag."""
        if training:
            return self.normalization(x)
        else:
            return x

    def compute_output_shape(self, input_shape):
        """Compute the output shape of the layer."""
        return input_shape

In [49]:
class TrainingTimeNormalization(tf.keras.layers.Layer):
    """Normalisation layer."""

    def __init__(self, axis=-1, **kwargs):
        super().__init__(**kwargs)
        self.normalization = tf.keras.layers.Normalization(axis=axis)

    def build(self, input_shape):
        """Build the normalization layer."""
        self.normalization.build(input_shape)

    def adapt(self, data):
        self.normalization.adapt(data)

    def call(self, x, training=None):
        if training:
            normalised_x = self.normalization(x)
            return normalised_x
        else:
            return x

In [50]:
norm_layer = TrainingTimeNormalization(axis=-1, name="preproc_norm")

In [51]:
data = np.random.random((100, 64, 64, 3))
norm_layer.adapt(data)

In [52]:
inputs = tf.keras.layers.Input([64, 64, 3], name="input_layer")
outputs = norm_layer(inputs)

model = tf.keras.Model(inputs, outputs)

In [53]:
model.save("test.keras")

In [54]:
model = tf.keras.models.load_model(
    "test.keras",
    custom_objects={
        "TrainingTimeNormalization": TrainingTimeNormalization(
            axis=-1, name="preproc_norm"
        ),
        "CloudsLayer": CloudsLayer(),
        "ConditionalNoiseLayer": ConditionalNoiseLayer([True, False, False]),
    },
    compile=False,
)

In [55]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 64, 64, 3)]       0         
                                                                 
 preproc_norm (TrainingTime  (None, 64, 64, 3)         7         
 Normalization)                                                  
                                                                 
Total params: 7 (32.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 7 (32.00 Byte)
_________________________________________________________________


In [4]:
        norm_layer = TrainingTimeNormalization(axis=-1, name="preproc_norm")
        norm_layer.adapt(model_trainer.data.x.train)

NameError: name 'model_trainer' is not defined

In [36]:
tf.keras.models.load_model(
        "w_last.keras",
        custom_objects={
         "ConditionalNoiseLayer": ConditionalNoiseLayer(model_trainer.data.x.xco2_noisy_chans),
           "CloudsLayer": CloudsLayer(model_trainer.data.x.clouds_training, model_trainer.data.x.cloudy_chans),
             "TrainingTimeNormalization": TrainingTimeNormalization(axis=-1, name="preproc_norm"),
         },
        compile=False,
  )

ValueError: Layer 'normalization_17' expected 0 variables, but received 3 variables during loading. Expected: []

In [18]:
tf.keras.models.save_model(model_trainer.model, "test_classic")

INFO:tensorflow:Assets written to: test_classic/assets


INFO:tensorflow:Assets written to: test_classic/assets


In [23]:
model_trainer.model.save("w_last.keras")

In [7]:
tf.saved_model.save(model_trainer.model, "test_saved_model")

INFO:tensorflow:Assets written to: test_saved_model/assets


INFO:tensorflow:Assets written to: test_saved_model/assets


In [15]:
loaded_model = tf.saved_model.load("test_saved_model")

In [16]:
loaded_model.summary()

AttributeError: '_UserObject' object has no attribute 'summary'